# Download Webpages

In [12]:
import requests
import urllib.request
import bs4
from bs4 import BeautifulSoup

import uuid

# Generate URL of 104 Website

In [4]:
def get_url_job_list(num_pages=50):
    """
    Generate url of 104 website.
        Sample url: 
              ['https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=1&mode=s&jobsource=2018indexpoc',
               'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=2&mode=s&jobsource=2018indexpoc',
               'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=3&mode=s&jobsource=2018indexpoc',
               'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=10&mode=s&jobsource=2018indexpoc',
               'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=11&mode=s&jobsource=2018indexpoc']

    Return: return url lists
    """
    
    url_job_list = []
    
    for index in range(1, num_pages+1):
        url = 'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page={}&mode=s&jobsource=2018indexpoc'.format(index)
        url_job_list.append(url)
        
        
    return url_job_list


url_job_list = get_url_job_list()

In [7]:
url_job_list[:5]

['https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=1&mode=s&jobsource=2018indexpoc',
 'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=2&mode=s&jobsource=2018indexpoc',
 'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=3&mode=s&jobsource=2018indexpoc',
 'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=4&mode=s&jobsource=2018indexpoc',
 'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=machine%20learning&order=1&asc=0&page=5&mode=s&jobsource=2018indexpoc']

# Declare Parser Class

In [19]:
class ListParser():
    def __init__(self, url):
        
        response = requests.get(url, timeout=10)
        self.content = BeautifulSoup(response.content, "html.parser")
        assert self.content is not None, 'Error: Empty content.'
        
        self.num_list = 0
        
        
    def get_lists(self):
        self.num_list = 0
        
        for element in self.content.findAll('article', attrs={'class': ['job-mode', 'js-job-item']}):
            if 'b-block--ad' in element.attrs['class']:
                continue
            else:
                self.num_list += 1
                yield element
        
        
class EntryParser():
    def __init__(self, element):
        """
        Args:
            element (bs4.element): element in BeautifulSoup
        """
        
        self.element = element
        
        
    def get_url(self):
        job = element.find('a', attrs={'class': 'js-job-link'})
        
        return job['href']
    
    
    def get_filename(self):
        str_uuid = uuid.uuid4()
        
        filename = '{}_{}_{}'.format(element['data-cust-name'], element['data-job-name'], str_uuid)
        return filename
    

# Main Program

In [21]:

for url in url_job_list[:5]:
    parser = ListParser(url)
    
    for element in parser.get_lists():
        entry = EntryParser(element)
        
        print(entry.get_url())
#         save_page(entry.get_url(), entry.get_filename())

//www.104.com.tw/job/6lng1?jobsource=2018indexpoc
//www.104.com.tw/job/6arae?jobsource=2018indexpoc
//www.104.com.tw/job/6heww?jobsource=2018indexpoc
//www.104.com.tw/job/46izo?jobsource=2018indexpoc
//www.104.com.tw/job/6arac?jobsource=2018indexpoc
//www.104.com.tw/job/6mst6?jobsource=2018indexpoc
//www.104.com.tw/job/5ezff?jobsource=2018indexpoc
//www.104.com.tw/job/6c1fr?jobsource=2018indexpoc
//www.104.com.tw/job/6luwg?jobsource=2018indexpoc
//www.104.com.tw/job/6izok?jobsource=2018indexpoc
//www.104.com.tw/job/5qrgx?jobsource=2018indexpoc
//www.104.com.tw/job/6g769?jobsource=2018indexpoc
//www.104.com.tw/job/6jiya?jobsource=2018indexpoc
//www.104.com.tw/job/64ki1?jobsource=2018indexpoc
//www.104.com.tw/job/5rm2f?jobsource=2018indexpoc
//www.104.com.tw/job/6lem2?jobsource=2018indexpoc
//www.104.com.tw/job/6mq24?jobsource=2018indexpoc
//www.104.com.tw/job/6af3o?jobsource=2018indexpoc
//www.104.com.tw/job/6aipb?jobsource=2018indexpoc
//www.104.com.tw/job/5xblu?jobsource=2018indexpoc
